# 🛡️ MedGuard
## AI-Powered Medical Misinformation Defense System
### MedGemma Impact Challenge — Kaggle 2026

> *2 billion people receive their health advice on WhatsApp
> before they ever see a doctor. MedGuard fights back.*

---

Built with Google MedGemma 4B-IT
Multimodal · Multilingual · Edge AI · Clinical Triage Ready

## 📖 The Problem

**220 million people. 1 doctor per 1,000 patients.
78% of WhatsApp health content is unverified.**

During COVID-19, Pakistanis drank Dettol, stopped
insulin, and abandoned chemotherapy — because of
forwarded messages. Not because they were careless.
Because they had no trusted alternative.

MedGuard is that alternative.

| Statistic | Value |
|---|---|
| Pakistan population | 220 million |
| WhatsApp daily users | 70%+ |
| Unverified health content | 78% |
| Doctors per 1,000 patients | <1 (rural) |
| Healthcare spend per capita | $38/year |
| Doctors who have emigrated | 40% |

In [ ]:
%%writefile medguard_core.py

import os, sys, subprocess, re, json, base64, io, contextlib
from io import BytesIO
from PIL import Image
from huggingface_hub import hf_hub_download

# ── 1. HARDWARE CHECK ─────────────────────────────────────────────────────────
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True)
    HAS_GPU = result.returncode == 0
except:
    HAS_GPU = False

# ── 2. DEPENDENCIES ───────────────────────────────────────────────────────────
try:
    import llama_cpp
    import argostranslate.package
    import argostranslate.translate
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'argostranslate', 'huggingface_hub', 'pillow'])
    if HAS_GPU:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'llama-cpp-python', '--extra-index-url', 'https://abetlen.github.io/llama-cpp-python/whl/cu121'])
    else:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'llama-cpp-python', '--extra-index-url', 'https://abetlen.github.io/llama-cpp-python/whl/cpu'])
    import llama_cpp
    import argostranslate.package
    import argostranslate.translate

from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava15ChatHandler

# ── 3. MODEL LOADING ──────────────────────────────────────────────────────────
def load_edge_model():
    text_gguf = hf_hub_download(repo_id="unsloth/medgemma-4b-it-GGUF", filename="medgemma-4b-it-Q4_K_M.gguf")
    vision_gguf = hf_hub_download(repo_id="kelkalot/medgemma-4b-it-GGUF", filename="mmproj-medgemma-4b-it-F16.gguf")

    chat_handler = Llava15ChatHandler(clip_model_path=vision_gguf)

    return Llama(
        model_path=text_gguf,
        chat_handler=chat_handler,
        chat_format="gemma",
        n_ctx=4096,
        n_threads=os.cpu_count() or 4,
        n_gpu_layers=-1 if HAS_GPU else 0,
        verbose=False
    )



Overwriting medguard_core.py


In [ ]:
# ── 4. TRANSLATION ────────────────────────────────────────────────────────────
def ensure_language_downloaded(target_code):
    if target_code == "en": return
    installed = argostranslate.package.get_installed_packages()
    if any(p.to_code == target_code and p.from_code == 'en' for p in installed): return
    argostranslate.package.update_package_index()
    available = argostranslate.package.get_available_packages()
    try:
        package = next(filter(lambda x: x.from_code == 'en' and x.to_code == target_code, available))
        argostranslate.package.install_from_path(package.download())
    except: pass

def translate_text(text, target_code):
    if target_code == "en" or not text or text == "N/A": return text
    try: return argostranslate.translate.translate(text, "en", target_code)
    except: return text


## 💡 How MedGuard Works

**You Input:**

| Field | Example |
|---|---|
| Health Claim | "Jeera water cures diabetes" |
| Age | 70 |
| Gender | Male |
| Medical History | Type 2 Diabetes, on Insulin |
| Symptoms | High blood sugar, fatigue |
| Image (optional) | X-Ray or wound photo |

**MedGuard Returns:**

⚖️ **Verdict** — True / False / Mixed / Insufficient Evidence

🚨 **Danger Score**
  🔴 High   = could cause death or serious harm
  🟡 Medium = risk if acted upon without guidance
  🟢 Low    = ineffective but not dangerous

🧠 **Medical Reasoning** — clinically calibrated to
  THIS patient's profile, age, history, and symptoms.
  Image findings folded in automatically.

👨‍⚕️ **Doctor Triage Note** — 3-line clinical summary.
  Physician reads it in 10 seconds and knows exactly
  what to check and how urgent it is.

🌍 **Your Language** — English, Urdu, Arabic,
  Hindi, French, or Spanish. Fully offline.

In [ ]:
# ── 5. JSON PARSER ────────────────────────────────────────────────────────────
def fix_json(raw):
    raw = re.sub(r'```json|```', '', raw).strip()
    raw = re.sub(r':\s*true\b',   ': "True"',   raw, flags=re.IGNORECASE)
    raw = re.sub(r':\s*false\b',  ': "False"',  raw, flags=re.IGNORECASE)
    raw = re.sub(r':\s*high\b',   ': "High"',   raw, flags=re.IGNORECASE)
    raw = re.sub(r':\s*medium\b', ': "Medium"', raw, flags=re.IGNORECASE)
    raw = re.sub(r':\s*low\b',    ': "Low"',    raw, flags=re.IGNORECASE)
    return raw

def parse_output(raw):
    try: return json.loads(fix_json(raw))
    except: pass
    try:
        match = re.search(r'\{.*?\}', raw, re.DOTALL)
        if match: return json.loads(fix_json(match.group()))
    except: pass

    def extract(p, t, d='N/A'):
        m = re.search(p, t, re.IGNORECASE | re.DOTALL)
        return m.group(1).strip() if m else d

    return {
        'claim':          extract(r'"claim"\s*:\s*"([^"]+)"',          raw),
        'verdict':        extract(r'"verdict"\s*:\s*["]?([^",}\n]+)',   raw),
        'reason':         extract(r'"reason"\s*:\s*"([^"]+)"',         raw),
        'doctor_summary': extract(r'"doctor_summary"\s*:\s*"([^"]+)"', raw),
        'risk_level':     extract(r'"risk_level"\s*:\s*["]?([^",}\n]+)',raw),
        'confidence':     extract(r'"confidence"\s*:\s*["]?([0-9.]+)', raw, '0.0'),
    }

# ── 6. CORE ANALYSIS ──────────────────────────────────────────────────────────
def analyze(model, user_claim, sym, history, image_path=None, target_lang="en"):
    ensure_language_downloaded(target_lang)

    raw_prompt = f"""<bos><start_of_turn>user
You are an expert Clinical Triage Specialist and Medical Fact-Checker.
Evaluate the patient's claim against established medical science.
Return ONLY a valid JSON object. No extra text, no markdown outside the JSON.

SCHEMA INSTRUCTIONS:
- 'claim': The health claim being evaluated.
- 'verdict': Strictly one of: True | False | Mixed | Insufficient Evidence
- 'reason': Provide a deep, scientific explanation of the physiological mechanisms or lack thereof. Explain clinically WHY the claim is true or false.
- 'doctor_summary': Write a professional clinical triage note. Must include: 1) Patient profile & symptoms, 2) The specific misinformation risk, and 3) Immediate clinical action items or red flags.
- 'risk_level': Low | Medium | High
- 'confidence': 0.0-1.0

EXAMPLE:
{{"claim":"Drinking bleach cures covid","verdict":"False","reason":"Sodium hypochlorite (bleach) is a caustic, toxic chemical. Ingestion causes severe mucosal liquefaction, esophageal burns, and systemic toxicity. It possesses no systemic antiviral pharmacological properties and cannot neutralize SARS-CoV-2 in vivo.","doctor_summary":"Triage Note: Patient presents with fever and cough, expressing intent to use household bleach as an antiviral remedy. HIGH ALERT for potential toxic ingestion. Action: Assess immediately for airway compromise or GI distress, consult toxicology, and educate patient on evidence-based viral management.","risk_level":"High","confidence":"0.98"}}

NOW EVALUATE:
Symptoms: {sym}
Claim: "{user_claim}"
Patient: {history}

OUTPUT JSON:<end_of_turn>
<start_of_turn>model
"""

    if not image_path:
        response = model.create_completion(
            prompt=raw_prompt,
            max_tokens=350,
            temperature=0.0,
            stop=["<end_of_turn>", "eos_token"]
        )
        raw = response['choices'][0]['text']
    else:
        try:
            img = Image.open(image_path).convert("RGB")
            buffered = BytesIO()
            img.save(buffered, format="JPEG")
            image_b64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        except:
            image_b64 = None

        content = [
            {"type": "text", "text": raw_prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_b64}"}}
        ]

        sink = io.StringIO()
        with contextlib.redirect_stdout(sink):
            response = model.create_chat_completion(
                messages=[{"role": "user", "content": content}],
                max_tokens=350,
                temperature=0.0,
                repeat_penalty=1.3
            )
        raw = response['choices'][0]['message']['content']

    parsed = parse_output(raw)

    if target_lang != "en":
        parsed['reason']         = translate_text(parsed.get('reason', ''),         target_lang)
        parsed['doctor_summary'] = translate_text(parsed.get('doctor_summary', ''), target_lang)

    return parsed

## 🔬 Technical Architecture

Patient Profile + Claim + Optional Image
              ↓

    MedGemma 4B-IT (Multimodal Foundation Model)
    GGUF Q4_K_M — runs on 4GB RAM, no GPU required
    Native Gemma prompt format for reliable output
              ↓

    Structured JSON Output
    3-Layer Parsing Pipeline
    (strict → regex → field-by-field fallback)
              ↓

    ArgosTranslate — 100% Offline (6 languages)
    No API calls. Works with zero internet.
              ↓

    Streamlit UI — Local or ngrok demo

**Stack:**
- Model: MedGemma 4B-IT (GGUF Q4_K_M)
- Runtime: llama.cpp — CPU + GPU auto-detection
- Quantization: 4-bit — ~4GB RAM, ARM compatible
- Translation: ArgosTranslate (fully offline)
- UI: Streamlit + ngrok (Colab demo)
- Edge: Zero cloud dependency after model download

In [ ]:
from medguard_core import load_edge_model, analyze

if 'model' not in globals():
    model = load_edge_model()

def print_result(result, title="MEDGUARD"):
    risk_emoji    = {"High":"🔴","Medium":"🟡","Low":"🟢"}.get(result.get('risk_level',''), "⚪")
    verdict_emoji = {"True":"✅","False":"❌","Mixed":"⚠️","Insufficient Evidence":"🔍"}.get(result.get('verdict',''), "❓")
    print("\n" + "="*50)
    print(f"        🛡️  {title}")
    print("="*50)
    print(f"📋 Claim:       {result.get('claim')}")
    print(f"⚖️  Verdict:     {verdict_emoji} {result.get('verdict')}")
    print(f"🧠 Reason:      {result.get('reason')}")
    print(f"👨‍⚕️  Doctor Note: {result.get('doctor_summary')}")
    print(f"{risk_emoji} Risk Level:  {result.get('risk_level')}")
    print(f"📊 Confidence:  {result.get('confidence')}")


result1 = analyze(
    model       = model,
    user_claim  = "Drinking Dettol to cure cancer",
    sym         = "Joint stiffness, mild nausea, fatigue",
    history     = "55-year-old Female, Rheumatoid Arthritis",
    target_lang = "en"
)

print_result(result1, "Test Case: Dettol Analysis")

/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<bos>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(



        🛡️  Test Case: Dettol Analysis
📋 Claim:       Drinking Dettol to cure cancer
⚖️  Verdict:     ❌ False
🧠 Reason:      Dettol is a disinfectant containing chloroxylenol, a broad-spectrum antimicrobial agent. It is not a medication and has no known mechanism of action to treat cancer. Cancer is a complex disease involving genetic mutations and uncontrolled cell growth, which are not affected by chloroxylenol. Ingestion of Dettol can cause gastrointestinal irritation, nausea, vomiting, and potentially more severe systemic effects depending on the amount ingested. It is not a cure for cancer.
👨‍⚕️  Doctor Note: Triage Note: Patient presents with joint stiffness, mild nausea, and fatigue, reporting the use of Dettol to treat cancer. This is a dangerous misconception. Dettol is a disinfectant and has no known efficacy against cancer. The patient's symptoms could be related to their rheumatoid arthritis or other underlying conditions. Immediate clinical action: Assess for any signs of

In [ ]:
# ── CASE 1 ────────────────────────────────────
result1 = analyze(
    model       = model,
    user_claim  = "Drinking Dettol cures COVID-19",
    sym         = "Fever, cough, shortness of breath",
    history     = "30-year-old Male, No chronic illness",
    target_lang = "en"
)
print_result(result1, "Case 1: Dettol Cures COVID-19")
# ── CASE 2 ────────────────────────────────────
result2 = analyze(
    model       = model,
    user_claim  = "Eating oranges gives you vitamin C and boosts immunity",
    sym         = "Fatigue, frequent colds",
    history     = "25-year-old Female, No known conditions",
    target_lang = "en"
)
print_result(result2, "Case 2: Oranges Give Vitamin C")


        🛡️  Case 1: Dettol Cures COVID-19
📋 Claim:       Drinking Dettol cures COVID-19
⚖️  Verdict:     ❌ False
🧠 Reason:      Dettol (chloroxylenol) is an antiseptic and disinfectant, not a medication. It works by disrupting bacterial cell membranes. It has no known antiviral properties against SARS-CoV-2, the virus that causes COVID-19. Ingesting Dettol can cause severe gastrointestinal irritation, nausea, vomiting, and potentially more serious systemic effects depending on the amount ingested. There is no scientific evidence to support its use as a treatment for COVID-19.
👨‍⚕️  Doctor Note: Triage Note: Patient presents with fever, cough, and shortness of breath, expressing intent to use Dettol as a COVID-19 treatment. HIGH ALERT for potential toxic ingestion. Action: Assess immediately for airway compromise or GI distress, consult toxicology, and educate patient on evidence-based viral management.
🔴 Risk Level:  High
📊 Confidence:  0.99

        🛡️  Case 2: Oranges Give Vitamin C

In [ ]:
# ── CASE 3 ────────────────────────────────────
result3 = analyze(
    model       = model,
    user_claim  = "Drinking cold water causes cancer",
    sym         = "No symptoms",
    history     = "40-year-old Male, Generally healthy",
    target_lang = "en"
)
print_result(result3, "Case 3: Cold Water Causes Cancer")
# ── CASE 4 ────────────────────────────────────
result4 = analyze(
    model       = model,
    user_claim  = "Jeera water permanently cures diabetes",
    sym         = "High blood sugar, fatigue, frequent urination",
    history     = "70-year-old Male, Type 2 Diabetes on Insulin",
    target_lang = "en"
)
print_result(result4, "Case 4: Jeera Water Cures Diabetes")


        🛡️  Case 3: Cold Water Causes Cancer
📋 Claim:       Drinking cold water causes cancer
⚖️  Verdict:     ❌ False
🧠 Reason:      There is no scientific evidence to support the claim that drinking cold water causes cancer. Cancer is a complex disease with multiple contributing factors, including genetics, environmental exposures, and lifestyle choices. Cold water consumption does not have a known mechanism by which it could directly cause cancer. While some studies have explored the potential effects of cold water on the body, such as its impact on metabolism and inflammation, these effects are not linked to cancer development. The idea that cold water causes cancer is a misconception.
👨‍⚕️  Doctor Note: Triage Note: Patient presents with no reported symptoms. The claim that drinking cold water causes cancer is unsubstantiated by scientific evidence. While cold water can affect the body, it does not have a known mechanism to cause cancer. The patient's current health status is gen

In [ ]:
# ── CASE 5 ────────────────────────────────────
result5 = analyze(
    model       = model,
    user_claim  = "Turmeric milk cures pneumonia completely",
    sym         = "High fever, chest pain, difficulty breathing",
    history     = "60-year-old Female, Hypertension",
    target_lang = "en"
)
print_result(result5, "Case 5: Turmeric Cures Pneumonia")


        🛡️  Case 5: Turmeric Cures Pneumonia
📋 Claim:       Turmeric milk cures pneumonia completely
⚖️  Verdict:     ❌ False
🧠 Reason:      Pneumonia is an infection of the lungs, typically caused by bacteria, viruses, or fungi. While turmeric has anti-inflammatory and antioxidant properties, it is not a cure for pneumonia. Antibiotics are required to treat bacterial pneumonia, antivirals for viral pneumonia, and antifungals for fungal pneumonia. Turmeric milk may provide some symptomatic relief, but it will not eliminate the underlying infection. Complete cure requires appropriate medical treatment.
👨‍⚕️  Doctor Note: Triage Note: Patient presents with fever, chest pain, and difficulty breathing, suggestive of pneumonia. The patient is a 60-year-old female with hypertension. The claim that turmeric milk cures pneumonia completely is false. Pneumonia requires specific medical treatment based on the causative agent. Turmeric milk may provide some symptomatic relief, but it is not a su

## 📊 Results — 5 Live Test Cases

| Case | Verdict | Risk | Confidence |
|---|---|---|---|
| Drinking Dettol cures COVID-19 | ❌ False | 🔴 High | 99% |
| Eating oranges gives vitamin C | ✅ True | 🟢 Low | 95% |
| Cold water causes cancer | ❌ False | 🟢 Low | 99% |
| Jeera water cures diabetes | ❌ False | 🟡 Medium | 95% |
| Turmeric cures pneumonia | ❌ False | 🟡 Medium | 95% |

✅ All 5 cases correctly evaluated with high confidence.
🔴 Toxic claims (Dettol) flagged at maximum risk instantly.
⚠️ Nuanced cases (Jeera, Turmeric) appropriately rated Medium — not blanket rejected.

---

🌍 **If MedGuard reaches 1% of Pakistan's population:**
- → 2.2 million people fact-checked instantly
- → Dangerous remedies stopped before harm occurs
- → Doctors pre-briefed before patient consultation
- → Rural patients guided to care in their own language

In [ ]:
%%writefile app.py

import os
import streamlit as st
from PIL import Image

# ── Import from our Edge-Optimized core ─────────
from medguard_core import load_edge_model, analyze

LANGUAGES = {
    "English": "en", "Urdu": "ur", "Arabic": "ar",
    "Hindi": "hi", "French": "fr", "Spanish": "es"
}

# ── Page Config ────────────────────────────────
st.set_page_config(
    page_title="🛡️ MedGuard",
    page_icon="⚕️",
    layout="wide"
)

# ── CSS ────────────────────────────────────────
st.markdown("""
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700&display=swap');
* { font-family: 'Inter', sans-serif; }
.stApp { background-color: #f0f4f8; }
.medguard-header {
    background: linear-gradient(135deg, #0077b6, #023e8a);
    border-radius: 16px;
    padding: 36px 24px;
    text-align: center;
    margin-bottom: 28px;
    box-shadow: 0 8px 32px rgba(0,119,182,0.3);
}
.medguard-header h1 {
    color: #ffffff !important;
    font-size: 2.8em;
    font-weight: 700;
    margin: 0;
    letter-spacing: -1px;
}
.medguard-header .subtitle { color: #90e0ef; font-size:1em; margin-top:8px; }
.badge {
    display: inline-block;
    background: rgba(255,255,255,0.15);
    color: #ffffff;
    border: 1px solid rgba(255,255,255,0.3);
    border-radius: 20px;
    padding: 4px 14px;
    font-size: 0.75em;
    margin: 3px;
}
.section-card {
    background: #ffffff;
    border-radius: 14px;
    padding: 24px;
    margin-bottom: 18px;
    border: 1px solid #dbe4ee;
    box-shadow: 0 2px 12px rgba(0,0,0,0.06);
}
.section-title {
    color: #0077b6;
    font-size: 1em;
    font-weight: 700;
    text-transform: uppercase;
    letter-spacing: 1px;
    margin-bottom: 16px;
    padding-bottom: 8px;
    border-bottom: 2px solid #e0f0ff;
}
.verdict-card { border-radius:14px; padding:24px; margin:12px 0; text-align:center; }
.verdict-false  { background:linear-gradient(135deg,#fff0f0,#ffe0e0); border:2px solid #ff4444; }
.verdict-true   { background:linear-gradient(135deg,#f0fff4,#e0ffe8); border:2px solid #00c853; }
.verdict-mixed  { background:linear-gradient(135deg,#fffbf0,#fff3e0); border:2px solid #ffa500; }
.verdict-insufficient { background:linear-gradient(135deg,#f0f8ff,#e0f0ff); border:2px solid #0077b6; }
.verdict-label  { font-size:2em; font-weight:700; margin:0; }
.verdict-risk   { font-size:1em; font-weight:600; margin-top:8px; color:#333; }
.risk-high      { color:#cc0000; font-weight:700; }
.risk-medium    { color:#e65c00; font-weight:700; }
.risk-low       { color:#1b7a1b; font-weight:700; }
.reason-box {
    background:#f0f8ff;
    border-left:4px solid #0077b6;
    border-radius:0 10px 10px 0;
    padding:16px 20px;
    margin:12px 0;
    color:#1a1a2e;
    font-size:0.95em;
    line-height:1.6;
}
.doctor-box {
    background:#fffbf0;
    border-left:4px solid #e6a817;
    border-radius:0 10px 10px 0;
    padding:16px 20px;
    margin:12px 0;
    color:#1a1a2e;
    font-size:0.95em;
    line-height:1.6;
}
.patient-box {
    background:#f8faff;
    border-radius:10px;
    padding:16px 20px;
    border:1px solid #dbe4ee;
    margin:12px 0;
}
.patient-box p { margin:6px 0; color:#333 !important; font-size:0.92em; }
.patient-box b { color:#0077b6; }
.placeholder-box { text-align:center; padding:80px 20px; color:#a0b8d0; }
.placeholder-box h2 { font-size:4em; margin:0; }
.placeholder-box h3 { color:#7090b0; margin:12px 0; }
[data-testid="metric-container"] {
    background:#ffffff; border:1px solid #dbe4ee;
    border-radius:10px; padding:12px;
    box-shadow:0 2px 8px rgba(0,0,0,0.04);
}
[data-testid="metric-container"] label { color:#0077b6 !important; font-weight:600 !important; }
[data-testid="metric-container"] [data-testid="metric-value"] { color:#023e8a !important; font-weight:700 !important; }
.stTextArea textarea, .stTextInput input, .stNumberInput input {
    background:#f8faff !important; border:1.5px solid #b8d4f0 !important;
    border-radius:8px !important; color:#1a1a2e !important;
}
.stButton > button {
    background:#ffffff; color:#0077b6;
    border:1.5px solid #0077b6; border-radius:8px;
    font-weight:600; font-size:0.88em;
}
.stButton > button:hover { background:#e0f0ff; }
[data-testid="baseButton-primary"] {
    background:linear-gradient(135deg,#0077b6,#023e8a) !important;
    color:#ffffff !important; border:none !important;
    border-radius:10px !important; font-size:1em !important;
    font-weight:700 !important;
    box-shadow:0 4px 16px rgba(0,119,182,0.4) !important;
}
.stProgress > div > div {
    background:linear-gradient(90deg,#0077b6,#00b4d8) !important;
    border-radius:10px !important;
}
hr { border-color:#dbe4ee !important; }
label { color:#1a3a5c !important; font-weight:600 !important; }
</style>
""", unsafe_allow_html=True)

# ── Header ─────────────────────────────────────
st.markdown("""
<div class="medguard-header">
    <h1>🛡️ MedGuard</h1>
    <p class="subtitle">AI-Powered Medical Misinformation Fighter</p>
    <div style="margin-top:12px">
        <span class="badge">⚕️ MedGemma</span>
        <span class="badge">🌍 Multilingual</span>
        <span class="badge">🩺 Triage Ready</span>
        <span class="badge">🖼️ Image Analysis</span>
        <span class="badge">🚀 Edge AI (Local)</span>
    </div>
</div>
""", unsafe_allow_html=True)

# ── Load Model  ───────────
@st.cache_resource
def get_model():
    return load_edge_model()

with st.spinner("⏳ Loading MedGuard Edge AI..."):
    model = get_model()


# ══════════════════════════════════════════════
# UI LAYOUT
# ══════════════════════════════════════════════
col1, col2 = st.columns([1, 1], gap="large")

# ── LEFT ───────────────────────────────────────
with col1:
    st.markdown("""
    <div class="section-card">
        <div class="section-title">📋 Patient Information</div>
    </div>""", unsafe_allow_html=True)

    user_claim = st.text_area(
        "🔍 Health Claim to Verify",
        value=st.session_state.get("ex_claim", ""),
        placeholder="Paste a health claim, home remedy, or medical advice...",
        height=100
    )
    symptoms = st.text_area(
        "🤒 Patient Symptoms",
        value=st.session_state.get("ex_sym", ""),
        placeholder="e.g. fever, fatigue, chest pain",
        height=75
    )

    c1, c2 = st.columns(2)
    with c1:
        age    = st.number_input("🎂 Age",
                    min_value=1, max_value=120, value=st.session_state.get("ex_age", 45))
    with c2:
        gender_index = ["Male", "Female", "Other"].index(st.session_state.get("ex_gen", "Male")) if st.session_state.get("ex_gen") else 0
        gender = st.selectbox("⚧ Gender", ["Male", "Female", "Other"], index=gender_index)

    condition = st.text_input(
        "🏥 Medical History",
        value=st.session_state.get("ex_cond", "No known conditions")
    )

    language = st.selectbox(
        "🌍 Output Language",
        list(LANGUAGES.keys()),
        index=0
    )

    uploaded = st.file_uploader(
        "🖼️ Upload X-Ray or Image (Optional)",
        type=["png","jpg","jpeg"]
    )

    img_path = None
    image = None
    if uploaded:
        image = Image.open(uploaded).convert("RGB")
        img_path = "temp_scan.jpg"
        image.save(img_path)
        st.image(image, caption="📷 Uploaded Image", use_container_width=True)

    st.markdown("---")

    examples = {

    }

    cols = st.columns(2)
    for i, (label, (c, s, a, g, h)) in enumerate(examples.items()):
        if cols[i%2].button(label, use_container_width=True):
            st.session_state.update({"ex_claim":c, "ex_sym":s, "ex_age":a, "ex_gen":g, "ex_cond":h})
            st.rerun()

    st.markdown("<br>", unsafe_allow_html=True)
    analyze_btn = st.button(
        "🛡️ Run MedGuard Analysis",
        type="primary",
        use_container_width=True
    )

# ── RIGHT ──────────────────────────────────────
with col2:
    st.markdown("""
    <div class="section-card">
        <div class="section-title">📊 MedGuard Analysis Result</div>
    </div>""", unsafe_allow_html=True)

    if analyze_btn:
        if not user_claim.strip():
            st.error("⚠️ Please enter a health claim.")
        else:
            history = f"{age} Years Old {gender}, {condition}"
            target_lang_code = LANGUAGES[language]

            with st.spinner("🔬 MedGuard running local analysis..."):
                result = analyze(
                    model       = model,
                    user_claim  = user_claim,
                    sym         = symptoms,
                    history     = history,
                    image_path  = img_path,
                    target_lang = target_lang_code
                )

            # Extract
            claim          = result.get("claim",          user_claim)
            verdict        = result.get("verdict",        "N/A").strip().strip('"')
            reason         = result.get("reason",         "N/A")
            doctor_summary = result.get("doctor_summary", "N/A")
            risk_level     = result.get("risk_level",     "N/A").strip().strip('"')
            try:
                confidence = float(result.get("confidence", 0.0))
            except:
                confidence = 0.0

            # Style maps
            verdict_styles = {
                "True":                 ("✅ TRUE",                 "verdict-true",         "#00c853"),
                "False":               ("❌ FALSE",                "verdict-false",        "#ff4444"),
                "Mixed":               ("⚠️ MIXED",                "verdict-mixed",        "#ffa500"),
                "Insufficient Evidence":("🔍 INSUFFICIENT EVIDENCE","verdict-insufficient", "#0077b6"),
            }
            risk_styles = {
                "High":   ("🔴","risk-high"),
                "Medium": ("🟡","risk-medium"),
                "Low":    ("🟢","risk-low"),
            }

            v_label, v_css, v_color = verdict_styles.get(
                verdict, (f"❓ {verdict}","verdict-insufficient","#888"))
            r_emoji, r_css = risk_styles.get(
                risk_level, ("⚪","risk-low"))

            # Verdict card
            st.markdown(f"""
            <div class="verdict-card {v_css}">
                <p class="verdict-label" style="color:{v_color}">
                    {v_label}
                </p>
                <p class="verdict-risk">
                    <span class="{r_css}">
                        {r_emoji} Risk Level: {risk_level}
                    </span>
                </p>
            </div>""", unsafe_allow_html=True)

            # Metrics
            m1, m2, m3 = st.columns(3)
            m1.metric("⚖️ Verdict",    verdict)
            m2.metric("🚨 Risk",       risk_level)
            m3.metric("📊 Confidence", f"{confidence:.0%}")
            st.progress(confidence)

            # Patient box
            st.markdown(f"""
            <div class="patient-box">
                <p>👤 <b>Patient:</b> {history}</p>
                <p>🤒 <b>Symptoms:</b> {symptoms or "Not provided"}</p>
                <p>🖼️ <b>Image:</b> {"Provided ✅" if image else "Not provided"}</p>
                <p>📋 <b>Claim:</b> {claim}</p>
            </div>""", unsafe_allow_html=True)

            # Reason
            st.markdown("**🧠 Medical Reasoning:**")
            st.markdown(f"""
            <div class="reason-box">{reason}</div>
            """, unsafe_allow_html=True)

            # Doctor note
            st.markdown("**👨‍⚕️ Doctor Triage Note:**")
            st.markdown(f"""
            <div class="doctor-box">{doctor_summary}</div>
            """, unsafe_allow_html=True)

            st.markdown("---")
            st.caption(
                "⚠️ MedGuard is an AI fact-checking tool running strictly locally on device memory. Always consult a qualified doctor."
            )
    else:
        st.markdown("""
        <div class="placeholder-box">
            <h2>🛡️</h2>
            <h3>MedGuard Ready</h3>
            <p>100% Offline Edge Intelligence</p>
            <p>Enter a health claim and click analyze.</p>
        </div>""", unsafe_allow_html=True)

Overwriting app.py


Streamlit UI
===

In [ ]:
import os
import subprocess
import time
import sys

# ── 0. Install Missing Server Dependencies ─────────────
try:
    import streamlit
except ImportError:
    print("⚙️ Installing Streamlit...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "streamlit"])

try:
    from pyngrok import ngrok
except ImportError:
    print("⚙️ Installing pyngrok...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pyngrok"])
    from pyngrok import ngrok

# ── 1. Authenticate ngrok  ────────
try:
    from google.colab import userdata
    NGROK_TOKEN = userdata.get('grok')
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ Successfully authenticated ngrok with Colab Secret 'grok'")
except Exception as e:
    print(f"❌ ERROR: Could not find the key 'grok' in Colab Secrets! Error: {e}")
    sys.exit(1)

# ── 2. Clean up old instances ──────────────────────────
print("🧹 Cleaning up old Streamlit instances...")
subprocess.run(["pkill", "-f", "streamlit"], capture_output=True)
time.sleep(2)

# ── 3. Launch Streamlit  ─────
print("🚀 Booting MedGuard UI...")
subprocess.Popen([
    sys.executable, "-m", "streamlit", "run", "app.py",
    "--server.port",       "8501",
    "--server.headless",   "true",
    "--server.enableCORS", "false",
    "--theme.base",        "light",
    "--theme.primaryColor","#0077b6",
])

time.sleep(5)

# ── 4. Create Public Tunnel ────────────────────────────
tunnel = ngrok.connect(8501)
print("="*50)
print("        🛡️  MEDGUARD IS LIVE!")
print(f"🔗 App URL: {tunnel.public_url}")
print("="*50)

✅ Successfully authenticated ngrok with Colab Secret 'grok'
🧹 Cleaning up old Streamlit instances...
🚀 Booting MedGuard UI...


        🛡️  MEDGUARD IS LIVE!
🔗 App URL: https://melania-splintery-unallegorically.ngrok-free.dev
